In [2]:
import pandas as pd 

df = pd.read_csv("../VIP_Target/VIP_With_Target.csv")

In [3]:
df.head()

,기준년월,발급회원번호,증감율_이용건수_신용_전월,증감율_이용건수_신판_전월,증감율_이용건수_일시불_전월,증감율_이용건수_할부_전월,증감율_이용건수_CA_전월,증감율_이용건수_체크_전월,증감율_이용건수_카드론_전월,증감율_이용금액_신용_전월,...,기타면제카드수_B0M,카드신청건수,Life_Stage,최종카드발급경과월,Slope_Spend,Slope_Balance,Slope_Count,Churn_Score,Cluster_ID,Target
0,201812,SYN_0,-0.048162,-0.048162,-0.048162,0.0,0.0,0.000000,0.0,0.238954,...,0,0,5.자녀성장기(2),27,-11116.685714,7214.657143,-0.571429,4.2,2,0
1,201812,SYN_1000,0.016580,0.016580,0.016580,0.0,0.0,0.000000,0.0,0.279102,...,0,0,4.자녀성장기(1),50,-15564.771429,-17170.742857,-2.428571,13.5,1,0
2,201812,SYN_1005,-0.094536,-0.094536,-0.094536,0.0,0.0,-0.037096,0.0,0.214526,...,0,0,2.가족형성기,13,-65200.657143,-10257.428571,-0.057143,11.4,1,0
3,201812,SYN_1007,-0.008564,-0.008564,-0.008564,0.0,0.0,0.000000,0.0,0.221973,...,0,0,3.자녀출산기,5,-3197.514286,-52205.085714,-1.000000,7.2,1,0
4,201812,SYN_1009,0.017918,0.017918,0.017918,0.0,0.0,-0.132413,0.0,0.239055,...,0,0,5.자녀성장기(2),27,4787.828571,-44019.000000,0.657143,2.1,2,0


In [6]:
import numpy as np
import shap
import xgboost as xgb
from statsmodels.stats.outliers_influence import variance_inflation_factor

df = df.head(2000)
print(f"데이터 2,000개 로딩 완료! (남은 행 개수: {len(df)}개)")

# 2. 분석용 데이터 만들기 (X, y 분리)
# 정답(Target)이랑 식별자(회원번호 등)는 문제지(X)에서 빼야 해
drop_cols = ['Target', 'Cluster_ID', 'Churn_Score', '발급회원번호', '기준년월']
existing_drop = [c for c in drop_cols if c in df.columns]

# 숫자만 남기고, 빈칸은 0으로 채우기
X = df.drop(columns=existing_drop).select_dtypes(include=[np.number])
X = X.fillna(0)
y = df['Target']

print(f"분석할 변수 개수: {len(X.columns)}개")

데이터 2,000개 로딩 완료! (남은 행 개수: 1910개)
분석할 변수 개수: 793개


In [7]:
import xgboost as xgb
import shap

print("1/3. SHAP 중요도 먼저 계산 중...")

# 1. 모델 학습 (XGBoost)
model = xgb.XGBClassifier(n_estimators=100, random_state=42, use_label_encoder=False, eval_metric='logloss')
model.fit(X, y)

# 2. SHAP 값 뽑기
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)

# 3. 중요도 평균내기 (절대값의 평균)
if isinstance(shap_values, list):
    shap_importance = np.abs(shap_values[1]).mean(axis=0)
else:
    shap_importance = np.abs(shap_values).mean(axis=0)

# 표로 만들기
shap_df = pd.DataFrame({
    "Feature": X.columns,
    "SHAP_Importance": shap_importance
})

print("SHAP 계산 끝!")

1/3. SHAP 중요도 먼저 계산 중...


/usr/local/Caskroom/miniconda/base/envs/myenv/lib/python3.13/site-packages/xgboost/training.py:199: UserWarning: [14:24:26] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1764148644238/work/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


SHAP 계산 끝!


In [8]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

print("2/3. VIF (다중공선성) 계산 중...")

vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns

# 모든 변수의 VIF 점수 계산
vif_data["VIF_Score"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print("VIF 계산 끝!")

2/3. VIF (다중공선성) 계산 중...


/usr/local/Caskroom/miniconda/base/envs/myenv/lib/python3.13/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
/usr/local/Caskroom/miniconda/base/envs/myenv/lib/python3.13/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/usr/local/Caskroom/miniconda/base/envs/myenv/lib/python3.13/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


VIF 계산 끝!


In [9]:
# -------------------------------------------------------
# 4. 결과 합치기 및 저장 (SHAP 기준 내림차순 정렬)
# -------------------------------------------------------
print("3/3. 결과 합치고 저장 중...")

# 1. SHAP 표랑 VIF 표 합치기
final_result = pd.merge(shap_df, vif_data, on="Feature", how="left")

# ★ 여기서 정렬해! (SHAP 중요도가 높은 게 위로 오게 내림차순)
final_result = final_result.sort_values(by="SHAP_Importance", ascending=False)

# 2. 상위 10개 눈으로 확인 (중요한 순서대로 나오는지 봐봐)
print("\n[가장 중요한 변수 TOP 10]")
print(final_result.head(10))

# 3. 파일로 저장
save_name = "../VIP_Target/VIP_Feature_Analysis_SHAP_VIF_2000.csv"
final_result.to_csv(save_name, index=False, encoding='utf-8-sig')

print(f"\n🎉 저장 완료! '{save_name}' 파일을 열어봐. 가장 중요한 변수가 맨 위에 있을 거야!")

3/3. 결과 합치고 저장 중...

[가장 중요한 변수 TOP 10]
              Feature  SHAP_Importance    VIF_Score
790       Slope_Spend         4.400259    45.051136
791     Slope_Balance         1.766061     5.326149
792       Slope_Count         1.393407    57.383331
233     _1순위쇼핑업종_이용금액         0.175062  1184.890019
42         변동률_잔액_B1M         0.172772     6.399300
768      최종유효년월_신용_이용         0.157263    17.375047
266  할부금액_무이자_6M_R12M         0.131916          inf
437        연체입금원금_B5M         0.130637          inf
164       이용금액_신용_R3M         0.101726          inf
459          카드이용한도금액         0.093139    55.602949

🎉 저장 완료! '../VIP_Target/VIP_Feature_Analysis_SHAP_VIF_2000.csv' 파일을 열어봐. 가장 중요한 변수가 맨 위에 있을 거야!
